# ComplianceChatbot: End-to-End Demo

This notebook shows how to:
1. Install dependencies & configure API keys
2. Ingest FATF/VARA docs → chunks → embeddings → FAISS
3. Retrieve and answer with the RAG pipeline (`answer(query)`)
4. Evaluate grounding & citation quality with a small eval set

> **Teaching style**: short steps with clear **what/why** comments.


In [1]:
# 1) Setup: install requirements (run once in your environment)
!pip install -r ../requirements.txt


[notice] A new release of pip available: 22.2.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Configure keys (.env)
- Copy `.env.example` to `.env` and fill `OPENAI_API_KEY` (and optional `OPENAI_BASE_URL`).
- Choose your `OPENAI_MODEL` (default: `gpt-4o-mini`).

### Add documents
- Put your PDFs and Markdown files into `../docs/`.
- Examples: `FATF_Recommendation_16.pdf`, `VARA_Client_Money.md`.


In [4]:
# 2) Ingest the corpus
import sys, os  
import sys   # Open the system toolbox
sys.argv = ['']   # clears Jupyter's extra args or Erase Jupyter’s messy notes and give it a clean slate

from src.ingest import main    # Bring in the ingestion machine
main()   # Turn on the ingestion machine.

Loading documents from: C:\Projects\compliance_chatbot\docs
Built 293 chunks. Embedding...


C:\Users\Sajeed\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.16it/s]

Saving FAISS index + metadata...
Done.


In [6]:
# 3) Ask questions via the RAG pipeline
from src.rag import answer
from src.config import Paths, Settings

res = answer("What does FATF Recommendation 16 say about wire transfers?")
print(res.text)

- FATF Recommendation 16 aims to prevent terrorists and criminals from accessing wire payments by ensuring that basic information about the originator and beneficiary is available to law enforcement and financial institutions.
- It applies to both cross-border and domestic wire transfers, requiring structured information to facilitate identification and reporting of suspicious transactions.
- Countries should minimize thresholds for wire transfers to trace all payments effectively while considering financial inclusion.

In summary, FATF Recommendation 16 focuses on enhancing transparency in wire transfers to combat money laundering and terrorist financing by ensuring that essential information is readily accessible to relevant authorities and institutions.

> Quote:  + quotes[0] + 
> Quote:  + extra + 

Citations: FATF_Recommendations.pdf#p77; Explanatory_Note_R16.pdf#p15; FATF_Recommendations.pdf#p87; FATF_Recommendations.pdf#p78; Explanatory_Note_R16.pdf#p1


In [7]:
# Another example
res = answer("What are VARA's rules on Client Money?")
print(res.text)

- "Client Money" refers to all money owned by a client and held by a VASP, excluding fees and expenses due to the VASP.
- Client Money must be held in a "Client Account" at a Third-Party Bank and is not owned by the VASP.
- VASPs must ensure Client Money is identifiable, secure, and not used to satisfy obligations to other clients.
- Client Money must be deposited into a Client Account within one calendar day of receipt.
- VASPs cannot take ownership of Client Money except for placing it under trust for clients.

In summary, VARA's rules stipulate that Client Money must be securely held in designated accounts, cannot be used for other clients' obligations, and VASPs must maintain strict controls and records regarding its handling.

> Quote:  + quotes[0] + 
> Quote:  + extra + 

Citations: VARA_Compliance_Risk.pdf#p37; VARA_Client_Money.md#section-b-treatment-of-client-money; VARA_Compliance_Risk.pdf#p38; VARA_Compliance_Risk.pdf#p40; VARA_Client_Money.md#section-a-application-and-inter

### 4) Evaluate grounding & citation quality
We run a small automated check to ensure each answer has citations, at least one quote, and uses the guardrail when retrieval is weak.

In [ ]:
from src.eval import main as eval_main
eval_main()